In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import datetime as dt
import seaborn as sns
from scipy import stats
import math

In [2]:
import os
print(os.getcwd())

C:\Users\장세환\Desktop\BigPyson\sewhan


In [3]:
os.chdir("E:/bigcontest")
print(os.getcwd())

E:\bigcontest


# data set

In [5]:
train = pd.read_excel('./train_0922.xlsx')


In [8]:
train['노출(분)'] = train['노출(분)'].replace(0, math.nan)
train['노출(분)'] = train['노출(분)'].fillna(method='ffill')

In [9]:
test = pd.read_excel('./test_ver1.xlsx', header=1)

In [16]:
train['분류'] 

0        f1
1        f1
2        f1
3       g11
4       g11
       ... 
7508     a1
7509    i29
7510    i29
7511    i29
7512    i29
Name: 분류, Length: 7513, dtype: object

# 변수 통합

In [18]:
# 빈도수인코딩(위의 원핫+pca와 비교해보기)
def add_frequency_encoding(data,column): 
    #데이터프레임,열을 받아서 빈도수인코딩열을 추가해줌
    enc_nom = (data.groupby(column).size())/len(data)
    data['freq_encode_{}'.format(column)] = data[column].apply(lambda x:enc_nom[x])
    print("freq_encode column was added")

In [19]:
add_frequency_encoding(train,'분류')

freq_encode column was added


In [20]:
#모든 카테고리에 대해 따로따로 자동 변수생성
category_list = train['상품군'].unique().tolist()
for i in category_list:
    #카테고리에 해당하는 주문량제외 train데이터 할당   -> 주문량 포함되어있음
    globals()['x_{}'.format(i)] = train.loc[train['상품군']==i] 
    #카테고리에 해당하는 train의 주문량을 할당
    globals()['y_{}'.format(i)] = train['new주문량'].loc[train['상품군']==i]

In [23]:
train['freq_encode_분류'] # 빈도 인코딩

0       0.021296
1       0.021296
2       0.021296
3       0.058432
4       0.058432
          ...   
7508    0.040596
7509    0.011447
7510    0.011447
7511    0.011447
7512    0.011447
Name: freq_encode_분류, Length: 7513, dtype: float64

In [24]:
train.columns

Index(['Unnamed: 0', '방송일시', '노출(분)', '마더코드', '상품코드', '상품명', '상품군', '취급액',
       '날짜', '시간', '요일', '시', 'holiday', '월', 'holiday(includeSS)',
       '실제_서울_최고기온', '실제_서울_최저기온', '실제_서울_강수량(mm)', '실제_서울_평균풍속(m/s)',
       '실제_서울_날씨', '실제_수원_최고기온', '실제_수원_최저기온', '실제_수원_강수량(mm)',
       '실제_수원_평균풍속(m/s)', '실제_수원_날씨', '실제_파주_최고기온', '실제_파주_최저기온',
       '실제_파주_강수량(mm)', '실제_파주_평균풍속(m/s)', '실제_파주_날씨', '예보_서울_강수확률',
       '예보_서울_강수량', '예보_서울_하늘상태', '예보_서울_일최고기온', '예보_서울_일최저기온', '예보_일산_강수확률',
       '예보_일산_강수량', '예보_일산_일최고기온', '예보_일산_일최저기온', '예보_수원_강수확률', '예보_수원_강수량',
       '예보_수원_일최고기온', '예보_수원_일최저기온', '계절', '미세먼지(㎍/㎥)', '오존(ppm)',
       '이산화질소농도(ppm)', '일산화탄소농도(ppm)', '아황산가스농도(ppm)', '초미세먼지(㎍/㎥)', '분류',
       'Active Users', 'new판매단가', 'new주문량', 'prime_time', 'prime_day',
       'top_code', 'top_cat', 'x1_cat', 'x2_cat', 'top_real_weather',
       '실제_최고기온', '실제_최저기온', '실제_강수량', '실제_평균풍속', '세일', 'freq_encode_분류'],
      dtype='object')

# 모델 생성 및 학습

In [25]:
#모든 카테고리에 대해 따로따로 자동 변수생성
category_list = train['상품군'].unique().tolist()
for i in category_list:
    #카테고리에 해당하는 주문량제외 train데이터 할당   -> 주문량 포함되어있음
    globals()['x_{}'.format(i)] = train.loc[train['상품군']==i] 
    #카테고리에 해당하는 train의 주문량을 할당
    globals()['y_{}'.format(i)] = train['new주문량'].loc[train['상품군']==i]

# split

카테고리별 20분 단위 3갈 각각 피팅 

In [28]:
catlist = train['상품군'].unique()

In [29]:
#그룹내 방송행간의 시간차이(날짜-날짜) <= 해당그룹 최대 노출(분)인가? 
# 검토해서 분리하는 함수

def split_shoprow(df, time): 
    shoprow1 = pd.DataFrame()  # train
    shoprow2 = pd.DataFrame()  # validation 
    size = len(df)

    for i in range((size)-1):       
        if (df['방송일시'].iloc[i+1]-df['방송일시'].iloc[i] <= time):
            shoprow1 = shoprow1.append(df.iloc[i])
        else:
            shoprow2 = shoprow2.append(df.iloc[i])    
    i += 1
    shoprow2 = shoprow2.append(df.iloc[i])
            
    return shoprow1,shoprow2

## 카테고리별로 split하기

In [30]:
for cat in catlist:
    selec_cat = globals()['x_{}'.format(cat)]
    momcode_cat = selec_cat['마더코드'].unique()
    # 걍 한 카테고리에서 가장 노출시간이 긴 시간을 기준으로 해도 될듯?
    # 20분씩
    max_time = selec_cat['노출(분)'].max()
    criteria_time = pd.Timedelta('0 days 00:{}:00'.format(int(max_time)))
    T = pd.DataFrame()
    V = pd.DataFrame()
    
    for momcode in momcode_cat:
        mom = train[train['마더코드']==momcode]
        t, v = split_shoprow(mom, criteria_time) 
        T = T.append(t)
        V = V.append(v)
    
    globals()['{}_bigfold'.format(cat)] = T
    globals()['{}_smallfold'.format(cat)] = V

In [184]:
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [32]:
beauty_smallfold.columns

Index(['Active Users', 'Unnamed: 0', 'freq_encode_분류', 'holiday',
       'holiday(includeSS)', 'new주문량', 'new판매단가', 'prime_day', 'prime_time',
       'top_cat', 'top_code', 'top_real_weather', 'x1_cat', 'x2_cat', '계절',
       '날짜', '노출(분)', '마더코드', '미세먼지(㎍/㎥)', '방송일시', '분류', '상품군', '상품명', '상품코드',
       '세일', '시', '시간', '실제_강수량', '실제_서울_강수량(mm)', '실제_서울_날씨', '실제_서울_최고기온',
       '실제_서울_최저기온', '실제_서울_평균풍속(m/s)', '실제_수원_강수량(mm)', '실제_수원_날씨',
       '실제_수원_최고기온', '실제_수원_최저기온', '실제_수원_평균풍속(m/s)', '실제_최고기온', '실제_최저기온',
       '실제_파주_강수량(mm)', '실제_파주_날씨', '실제_파주_최고기온', '실제_파주_최저기온',
       '실제_파주_평균풍속(m/s)', '실제_평균풍속', '아황산가스농도(ppm)', '예보_서울_강수량', '예보_서울_강수확률',
       '예보_서울_일최고기온', '예보_서울_일최저기온', '예보_서울_하늘상태', '예보_수원_강수량', '예보_수원_강수확률',
       '예보_수원_일최고기온', '예보_수원_일최저기온', '예보_일산_강수량', '예보_일산_강수확률', '예보_일산_일최고기온',
       '예보_일산_일최저기온', '오존(ppm)', '요일', '월', '이산화질소농도(ppm)', '일산화탄소농도(ppm)',
       '초미세먼지(㎍/㎥)', '취급액'],
      dtype='object')

In [89]:
seungen_feature2 = ['노출(분)', 'holiday(includeSS)','실제_최고기온','실제_최저기온','실제_강수량','실제_평균풍속' , '예보_서울_강수확률', '예보_서울_강수량',
       '예보_서울_하늘상태', '예보_서울_일최고기온', '예보_서울_일최저기온', '예보_일산_강수확률', '예보_일산_강수량',
       '예보_일산_일최고기온', '예보_일산_일최저기온', '예보_수원_강수확률', '예보_수원_강수량', '예보_수원_일최고기온',
       '예보_수원_일최저기온', 'Active Users', 'new판매단가', 'prime_time',
       'prime_day', 'top_code', 'top_cat','x1_cat','x2_cat']




In [90]:
feature_selection= seungen_feature2

In [62]:

y_beauty_bigfold = beauty_bigfold['new주문량']
y_beauty_smallfold = beauty_smallfold['new주문량']

In [63]:
x_beauty_bigfold = beauty_bigfold[seungen_feature2]
x_beauty_smallfold = beauty_smallfold[seungen_feature2]

# Random Forest

In [64]:

from sklearn.ensemble import RandomForestRegressor
forest = RandomForestRegressor(random_state=1234)

In [91]:

for i in category_list:
  globals()['x_{}_bigfold'.format(i)] = globals()['{}_bigfold'.format(i)][feature_selection]
  globals()['x_{}_smallfold'.format(i)] = globals()['{}_smallfold'.format(i)][feature_selection]
  globals()['y_{}_bigfold'.format(i)] = globals()['{}_bigfold'.format(i)]['new주문량']
  globals()['y_{}_smallfold'.format(i)] = globals()['{}_smallfold'.format(i)]['new주문량']

mape_score_3gal = []
for i in category_list:
  category_data = globals()['x_{}_bigfold'.format(i)]
  category_label =  globals()['y_{}_bigfold'.format(i)]
  forest.fit(category_data,category_label)          # bigfold로 훈련시킴
  test_data = globals()['x_{}_smallfold'.format(i)]
  test_label =  globals()['y_{}_smallfold'.format(i)]
  predictions = forest.predict(test_data) 
  mape_score_3gal.append(mean_absolute_percentage_error(test_label, predictions)) # smallfold 데이터로 예측

In [92]:
for i in range(len(category_list)):
  print("RF MAPE score of test",category_list[i],":",mape_score_3gal[i])

RF MAPE score of test inner : 9.785157210115965
RF MAPE score of test cloth : 10.547171584113967
RF MAPE score of test elec : 26.901645431486248
RF MAPE score of test living : 13.405616644245471
RF MAPE score of test food : 5.308406816778787
RF MAPE score of test health : 8.52870905045054
RF MAPE score of test bedding : 12.356173105743649
RF MAPE score of test furniture : 25.8239978935915
RF MAPE score of test beauty : 7.599210988632526
RF MAPE score of test etc : 17.30755171904211
RF MAPE score of test life : 15.617103275946098


In [93]:
pd.Series(mape_score_3gal).mean()



13.925522156376987

seungen_feature2 = ['노출(분)', 'holiday(includeSS)','실제_최고기온','실제_최저기온','실제_강수량','실제_평균풍속' , '예보_서울_강수확률', '예보_서울_강수량',
       '예보_서울_하늘상태', '예보_서울_일최고기온', '예보_서울_일최저기온', '예보_일산_강수확률', '예보_일산_강수량',
       '예보_일산_일최고기온', '예보_일산_일최저기온', '예보_수원_강수확률', '예보_수원_강수량', '예보_수원_일최고기온',
       '예보_수원_일최저기온', 'Active Users', 'new판매단가', 'prime_time',
       'prime_day', 'top_code', 'top_cat','x1_cat','x2_cat','top_real_weather','freq_encode_분류']
-> 13.904164439370625 

seungen_feature2 = ['노출(분)', 'holiday(includeSS)','실제_최고기온','실제_최저기온','실제_강수량','실제_평균풍속' , '예보_서울_강수확률', '예보_서울_강수량',
       '예보_서울_하늘상태', '예보_서울_일최고기온', '예보_서울_일최저기온', '예보_일산_강수확률', '예보_일산_강수량',
       '예보_일산_일최고기온', '예보_일산_일최저기온', '예보_수원_강수확률', '예보_수원_강수량', '예보_수원_일최고기온',
       '예보_수원_일최저기온', 'Active Users', 'new판매단가', 'prime_time',
       'prime_day', 'top_code', 'top_cat','x1_cat','x2_cat','top_real_weather']
-> 13.878734898890759


seungen_feature = ['노출(분)', 'holiday(includeSS)','실제_최고기온','실제_최저기온','실제_강수량','실제_평균풍속' , '예보_서울_강수확률', '예보_서울_강수량',
       '예보_서울_하늘상태', '예보_서울_일최고기온', '예보_서울_일최저기온', '예보_일산_강수확률', '예보_일산_강수량',
       '예보_일산_일최고기온', '예보_일산_일최저기온', '예보_수원_강수확률', '예보_수원_강수량', '예보_수원_일최고기온',
       '예보_수원_일최저기온', 'Active Users', 'new판매단가', 'prime_time',
       'prime_day', 'top_code', 'top_cat']
->13.866119096556979


seungen_feature2 = ['노출(분)', 'holiday(includeSS)','실제_최고기온','실제_최저기온','실제_강수량','실제_평균풍속' , '예보_서울_강수확률', '예보_서울_강수량',
       '예보_서울_하늘상태', '예보_서울_일최고기온', '예보_서울_일최저기온', '예보_일산_강수확률', '예보_일산_강수량',
       '예보_일산_일최고기온', '예보_일산_일최저기온', '예보_수원_강수확률', '예보_수원_강수량', '예보_수원_일최고기온',
       '예보_수원_일최저기온', 'Active Users', 'new판매단가', 'prime_time',
       'prime_day', 'top_code', 'top_cat','x1_cat','x2_cat']

->13.863088620722529 -> 요거 쓰자!


# GB_AUTOML

In [122]:
import h2o
from h2o.estimators import H2OGradientBoostingEstimator
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O_cluster_uptime:,2 days 11 hours 28 mins
H2O_cluster_timezone:,Asia/Seoul
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.30.0.7
H2O_cluster_version_age:,2 months and 1 day
H2O_cluster_name:,H2O_from_python_占쎌삢占쎄쉭占쎌넎_s8b2y9
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,2.269 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


In [103]:
automl_feature = ['노출(분)', 'holiday(includeSS)','실제_최고기온','실제_최저기온','실제_강수량','실제_평균풍속' , '예보_서울_강수확률', '예보_서울_강수량',
       '예보_서울_하늘상태', '예보_서울_일최고기온', '예보_서울_일최저기온', '예보_일산_강수확률', '예보_일산_강수량',
       '예보_일산_일최고기온', '예보_일산_일최저기온', '예보_수원_강수확률', '예보_수원_강수량', '예보_수원_일최고기온',
       '예보_수원_일최저기온', 'Active Users', 'new판매단가', 'prime_time',
       'prime_day', 'top_code', 'top_cat','x1_cat','x2_cat','new주문량']

In [151]:
# split train, validation set
train= h2o.H2OFrame(beauty_bigfold[automl_feature])
valid= h2o.H2OFrame(beauty_smallfold[automl_feature ])

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [105]:
train.head()

�끂異�(遺�),holiday(includeSS),�떎�젣_理쒓퀬湲곗삩,�떎�젣_理쒖�湲곗삩,�떎�젣_媛뺤닔�웾,�떎�젣_�룊洹좏뭾�냽,�삁蹂�_�꽌�슱_媛뺤닔�솗瑜�,�삁蹂�_�꽌�슱_媛뺤닔�웾,�삁蹂�_�꽌�슱_�븯�뒛�긽�깭,�삁蹂�_�꽌�슱_�씪理쒓퀬湲곗삩,�삁蹂�_�꽌�슱_�씪理쒖�湲곗삩,�삁蹂�_�씪�궛_媛뺤닔�솗瑜�,�삁蹂�_�씪�궛_媛뺤닔�웾,�삁蹂�_�씪�궛_�씪理쒓퀬湲곗삩,�삁蹂�_�씪�궛_�씪理쒖�湲곗삩,�삁蹂�_�닔�썝_媛뺤닔�솗瑜�,�삁蹂�_�닔�썝_媛뺤닔�웾,�삁蹂�_�닔�썝_�씪理쒓퀬湲곗삩,�삁蹂�_�닔�썝_�씪理쒖�湲곗삩,Active Users,new�뙋留ㅻ떒媛�,prime_time,prime_day,top_code,top_cat,x1_cat,x2_cat,new二쇰Ц�웾
nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
20,0,26.1,17.2667,0.0666667,2,29.1892,1.44737,3.21622,26.4737,18,30.4054,1.44737,24.9474,17.4375,26.6892,1.44737,26.8947,17.4375,110345,11.1419,6,3,0,42,-0.00420563,0.00531476,5.81862
20,0,26.3667,18.3,0.0666667,1.66667,10.473,0.526316,1.94595,28.7368,18,10.1351,0.328947,28.1053,16.3125,9.52703,0.328947,29.4737,18,122905,11.1419,7,6,0,42,-0.00420563,0.00531476,5.58033
20,0,26.3667,18.3,0.0666667,1.66667,10.473,0.526316,1.94595,28.7368,18,10.1351,0.328947,28.1053,16.3125,9.52703,0.328947,29.4737,18,122905,11.1419,7,6,0,42,-0.00420563,0.00531476,5.94055
20,0,30.5333,18.9,0,1.7,6.14865,0,1.61486,31.8947,19.9375,6.08108,0,30.2105,18,4.39189,0,31.7368,18.875,128337,11.1419,3,3,0,42,-0.00420563,0.00531476,5.66079
20,0,30.5333,18.9,0,1.7,6.14865,0,1.61486,31.8947,19.9375,6.08108,0,30.2105,18,4.39189,0,31.7368,18.875,128337,11.1419,3,3,0,42,-0.00420563,0.00531476,6.0939
20,0,28.9333,20.9667,0.233333,1.4,27.7027,1.25,3.24324,27.7895,20.4375,25.5405,0.855263,27,20,30,1.65789,26.6316,20.75,130279,11.1419,8,6,0,42,-0.00420563,0.00531476,4.62412
20,0,28.9333,20.9667,0.233333,1.4,27.7027,1.25,3.24324,27.7895,20.4375,25.5405,0.855263,27,20,30,1.65789,26.6316,20.75,130279,11.1419,8,6,0,42,-0.00420563,0.00531476,5.5311
30,0,24.1,17.1333,13.2,1.36667,32.2297,2.17105,3.48649,26.9474,18.4375,32.0946,2.17105,25.8947,18,32.027,1.71053,27,18.4375,117841,10.9988,5,7,39,57,-0.00436986,0.00552503,6.31132
30,0,25.3333,18.6667,0.233333,1.53333,27.027,2.30263,3.0473,27.3684,19,26.0135,2.23684,26.2105,18.9375,24.1216,1.11842,27.4211,19,123763,10.9988,11,2,39,57,-0.00436986,0.00552503,6.30663


In [152]:
col=[]
for i in range(len(train.columns)):
    col.append('a_{}'.format(i))
   
train.columns=col    
valid.columns=col

In [108]:
valid.head()

a_0,a_1,a_2,a_3,a_4,a_5,a_6,a_7,a_8,a_9,a_10,a_11,a_12,a_13,a_14,a_15,a_16,a_17,a_18,a_19,a_20,a_21,a_22,a_23,a_24,a_25,a_26,a_27
nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
13,0,26.1,17.2667,0.0666667,2,29.1892,1.44737,3.21622,26.4737,18,30.4054,1.44737,24.9474,17.4375,26.6892,1.44737,26.8947,17.4375,110345,11.1419,6,3,0,42,-0.00420563,0.00531476,5.49025
20,0,26.3667,18.3,0.0666667,1.66667,10.473,0.526316,1.94595,28.7368,18,10.1351,0.328947,28.1053,16.3125,9.52703,0.328947,29.4737,18,122905,11.1419,7,6,0,42,-0.00420563,0.00531476,6.14546
20,0,30.5333,18.9,0,1.7,6.14865,0,1.61486,31.8947,19.9375,6.08108,0,30.2105,18,4.39189,0,31.7368,18.875,128337,11.1419,3,3,0,42,-0.00420563,0.00531476,6.13666
20,0,28.9333,20.9667,0.233333,1.4,27.7027,1.25,3.24324,27.7895,20.4375,25.5405,0.855263,27,20,30,1.65789,26.6316,20.75,130279,11.1419,8,6,0,42,-0.00420563,0.00531476,5.81884
30,0,24.1,17.1333,13.2,1.36667,32.2297,2.17105,3.48649,26.9474,18.4375,32.0946,2.17105,25.8947,18,32.027,1.71053,27,18.4375,117841,10.9988,5,7,39,57,-0.00436986,0.00552503,7.19634
30,0,25.3333,18.6667,0.233333,1.53333,27.027,2.30263,3.0473,27.3684,19,26.0135,2.23684,26.2105,18.9375,24.1216,1.11842,27.4211,19,123763,10.9988,11,2,39,57,-0.00436986,0.00552503,6.9384
23,0,26.3667,18.3,0.0666667,1.66667,10.473,0.526316,1.94595,28.7368,18,10.1351,0.328947,28.1053,16.3125,9.52703,0.328947,29.4737,18,122905,10.9988,6,6,39,57,-0.00436986,0.00552503,5.88782
30,1,30.3,16.7,0,1.83333,3.31081,0,1.33108,31.5789,19.75,1.95946,0,28.8421,18,3.10811,0,30.6316,18.5625,113784,10.9988,9,4,39,57,-0.00436986,0.00552503,7.11891
30,0,30.5333,18.9,0,1.7,6.14865,0,1.61486,31.8947,19.9375,6.08108,0,30.2105,18,4.39189,0,31.7368,18.875,123984,10.9988,5,3,39,57,-0.00436986,0.00552503,7.03204


In [159]:
valid[1:,:]#첫번째 행에 생긴 nan을 제거

a_0,a_1,a_2,a_3,a_4,a_5,a_6,a_7,a_8,a_9,a_10,a_11,a_12,a_13,a_14,a_15,a_16,a_17,a_18,a_19,a_20,a_21,a_22,a_23,a_24,a_25,a_26,a_27
13,0,26.1,17.2667,0.0666667,2,29.1892,1.44737,3.21622,26.4737,18,30.4054,1.44737,24.9474,17.4375,26.6892,1.44737,26.8947,17.4375,110345,11.1419,6,3,0,42,-0.00420563,0.00531476,5.49025
20,0,26.3667,18.3,0.0666667,1.66667,10.473,0.526316,1.94595,28.7368,18,10.1351,0.328947,28.1053,16.3125,9.52703,0.328947,29.4737,18,122905,11.1419,7,6,0,42,-0.00420563,0.00531476,6.14546
20,0,30.5333,18.9,0,1.7,6.14865,0,1.61486,31.8947,19.9375,6.08108,0,30.2105,18,4.39189,0,31.7368,18.875,128337,11.1419,3,3,0,42,-0.00420563,0.00531476,6.13666
20,0,28.9333,20.9667,0.233333,1.4,27.7027,1.25,3.24324,27.7895,20.4375,25.5405,0.855263,27,20,30,1.65789,26.6316,20.75,130279,11.1419,8,6,0,42,-0.00420563,0.00531476,5.81884
30,0,24.1,17.1333,13.2,1.36667,32.2297,2.17105,3.48649,26.9474,18.4375,32.0946,2.17105,25.8947,18,32.027,1.71053,27,18.4375,117841,10.9988,5,7,39,57,-0.00436986,0.00552503,7.19634
30,0,25.3333,18.6667,0.233333,1.53333,27.027,2.30263,3.0473,27.3684,19,26.0135,2.23684,26.2105,18.9375,24.1216,1.11842,27.4211,19,123763,10.9988,11,2,39,57,-0.00436986,0.00552503,6.9384
23,0,26.3667,18.3,0.0666667,1.66667,10.473,0.526316,1.94595,28.7368,18,10.1351,0.328947,28.1053,16.3125,9.52703,0.328947,29.4737,18,122905,10.9988,6,6,39,57,-0.00436986,0.00552503,5.88782
30,1,30.3,16.7,0,1.83333,3.31081,0,1.33108,31.5789,19.75,1.95946,0,28.8421,18,3.10811,0,30.6316,18.5625,113784,10.9988,9,4,39,57,-0.00436986,0.00552503,7.11891
30,0,30.5333,18.9,0,1.7,6.14865,0,1.61486,31.8947,19.9375,6.08108,0,30.2105,18,4.39189,0,31.7368,18.875,123984,10.9988,5,3,39,57,-0.00436986,0.00552503,7.03204
30,0,31.9667,17.6,0,1.33333,23.5811,2.23684,2.79054,30.3158,21,24.0541,2.23684,29.1579,18.875,25,2.31579,30.6842,20.4375,137688,10.9988,3,7,39,57,-0.00436986,0.00552503,6.69592


In [153]:
# Set the predictors and response; set the response as a factor:
predictors = train.columns
response = 'a_27'

In [193]:
# Build and train the model:
pros_gbm = H2OGradientBoostingEstimator( seed=1111 )
                                        
pros_gbm.train(x=predictors, y=response, training_frame=train)


gbm Model Build progress: |███████████████████████████████████████████████| 100%


In [194]:

# Generate predictions on a test set (if necessary):
predictions  = pros_gbm .predict(valid[1:,:])

gbm prediction progress: |████████████████████████████████████████████████| 100%


In [165]:
len(y_beauty_smallfold)

72

In [163]:
len(valid)

73

In [195]:
a=h2o.as_list(predictions)

In [180]:
y_beauty_smallfold

215     5.490254
572     6.145460
920     6.136662
1256    5.818835
335     7.196336
          ...   
7423    6.993408
7015    5.428126
7353    4.939136
7108    5.780717
7407    7.127991
Name: new주문량, Length: 72, dtype: float64

In [196]:
mape = mean_absolute_percentage_error(y_beauty_smallfold, a)
mape

11.569696270019447

In [204]:
for i in category_list:
  # train  
  globals()['x_{}_bigfold'.format(i)] = h2o.H2OFrame(globals()['{}_bigfold'.format(i)][automl_feature])
  globals()['x_{}_bigfold'.format(i)].columns=col

  # valid
  globals()['x_{}_smallfold'.format(i)] =h2o.H2OFrame( globals()['{}_smallfold'.format(i)][automl_feature])
  globals()['x_{}_smallfold'.format(i)].columns=col
  # traget
  globals()['y_{}_smallfold'.format(i)] = globals()['{}_smallfold'.format(i)]['new주문량']


Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |███████████

In [211]:
# Set the predictors and response; set the response as a factor:
predictors = train.columns
response = 'a_27'

mape_score_3gal = []
for i in category_list:
  category_data = globals()['x_{}_bigfold'.format(i)]

  pros_gbm = H2OGradientBoostingEstimator(seed=1111)  
  pros_gbm.train(x=predictors, y=response, training_frame=category_data)  
         # bigfold로 훈련시킴
  test_data = globals()['x_{}_smallfold'.format(i)][1:,:]
  print(i)
  test_label =  globals()['y_{}_smallfold'.format(i)]
  predictions = pros_gbm.predict(test_data) 
  mape_score_3gal.append(mean_absolute_percentage_error(test_label, h2o.as_list(predictions))) # smallfold 데이터로 예측

gbm Model Build progress: |███████████████████████████████████████████████| 100%
inner
gbm prediction progress: |████████████████████████████████████████████████| 100%
gbm Model Build progress: |███████████████████████████████████████████████| 100%
cloth
gbm prediction progress: |████████████████████████████████████████████████| 100%
gbm Model Build progress: |███████████████████████████████████████████████| 100%
elec
gbm prediction progress: |████████████████████████████████████████████████| 100%
gbm Model Build progress: |███████████████████████████████████████████████| 100%
living
gbm prediction progress: |████████████████████████████████████████████████| 100%
gbm Model Build progress: |███████████████████████████████████████████████| 100%
food
gbm prediction progress: |████████████████████████████████████████████████| 100%
gbm Model Build progress: |███████████████████████████████████████████████| 100%
health
gbm prediction progress: |███████████████████████████████████████████████

In [212]:
for i in range(len(category_list)):
  print("GB MAPE score of test",category_list[i],":",mape_score_3gal[i])

GB MAPE score of test inner : 19.445490206046188
GB MAPE score of test cloth : 27.84415869219343
GB MAPE score of test elec : 69.11613009338534
GB MAPE score of test living : 27.73151744484228
GB MAPE score of test food : 7.98659929697134
GB MAPE score of test health : 18.5448063317454
GB MAPE score of test bedding : 16.812954988468327
GB MAPE score of test furniture : 53.64432197359284
GB MAPE score of test beauty : 11.569696270019447
GB MAPE score of test etc : 71.20284568110597
GB MAPE score of test life : 40.104776460974456


# Random Forest_AUTOML

In [213]:
import h2o
from h2o.estimators import H2ORandomForestEstimator
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O_cluster_uptime:,2 days 13 hours 37 mins
H2O_cluster_timezone:,Asia/Seoul
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.30.0.7
H2O_cluster_version_age:,2 months and 1 day
H2O_cluster_name:,H2O_from_python_占쎌삢占쎄쉭占쎌넎_s8b2y9
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,2.250 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


In [219]:
for i in category_list:
  category_data = globals()['x_{}_bigfold'.format(i)]

  cars_drf = H2ORandomForestEstimator()
                            
  cars_drf.train(x=predictors, y=response, training_frame=category_data)  
         # bigfold로 훈련시킴
  test_data = globals()['x_{}_smallfold'.format(i)][1:,:]
  test_label =  globals()['y_{}_smallfold'.format(i)]
  predictions = cars_drf.predict(test_data) 
  mape_score_3gal.append(mean_absolute_percentage_error(test_label, h2o.as_list(predictions))) # smallfold 데이터로 예측

drf Model Build progress: |███████████████████████████████████████████████| 100%
drf prediction progress: |████████████████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████| 100%
drf prediction progress: |████████████████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████| 100%
drf prediction progress: |████████████████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████| 100%
drf prediction progress: |████████████████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████| 100%
drf prediction progress: |████████████████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████| 100%
drf prediction progress: |████████████████████████████████████████████████| 100%
drf Model Build progress: |█

In [220]:
for i in range(len(category_list)):
  print("RF_automl MAPE score of test",category_list[i],":",mape_score_3gal[i])

RF_automl MAPE score of test inner : 19.593582140282916
RF_automl MAPE score of test cloth : 27.866840510362383
RF_automl MAPE score of test elec : 69.94063170960011
RF_automl MAPE score of test living : 27.552901056399836
RF_automl MAPE score of test food : 8.075594674654242
RF_automl MAPE score of test health : 18.50649098630209
RF_automl MAPE score of test bedding : 16.888047654391123
RF_automl MAPE score of test furniture : 53.99692914547229
RF_automl MAPE score of test beauty : 11.483056823504354
RF_automl MAPE score of test etc : 71.57758872860921
RF_automl MAPE score of test life : 39.37394720958795
